### Query ORCID for works authored by a person

This notebook queries the [ORCID API](https://pub.orcid.org/v3.0/) to retrieve all works listed in a person's ORCID profile. It takes an ORCID URL or iD as input to retrieve the ORCID profile of a person and the works listed on it. From the list of works we output their respective DOIs.

In [ ]:
# Prerequisites:
!pip install requests
import requests                        # dependency to make HTTP calls
!pip install python-benedict
from benedict import benedict          # dependency for dealing with json

The input for the query is an ORCID URL or ORCID iD, e.g. '`https://orcid.org/0000-0003-2499-7741`' or '`0000-0003-2499-7741`'.

In [2]:
# input parameter
example_orcid="https://orcid.org/0000-0003-2499-7741"

We use it to query ORCID for the person's metadata and all works connected to them.

In [3]:
# URL for ORCID API
ORCID_RECORD_API = "https://pub.orcid.org/v3.0/"

# query ORCID for an ORCID record
def query_orcid_for_record(orcid_id):

    response = requests.get(url=requests.utils.requote_uri(ORCID_RECORD_API + orcid_id),
                          headers={'Accept': 'application/json'})
    response.raise_for_status()
    result=response.json()
    return result


#-- example execution
orcid_id=example_orcid.replace("https://orcid.org/", "")
orcid_record=query_orcid_for_record(orcid_id)
# uncomment next lines to see complete metadata for given ORCID
#import pprint
#pprint.pprint(orcid_data)

From the complete ORCID metadata we extract the works section and print out title and DOI of each first work-summary (the first item in a personal information section has the highest [display index](https://info.orcid.org/documentation/integration-guide/orcid-record/#Display_index)).

In [4]:
# extract works section from ORCID profile
def extract_works_section(orcid_record):
    orcid_dict=benedict.from_json(orcid_record)
    works=orcid_dict.get('activities-summary.works.group') or []
    return works

# for each work in the work section: extract title and DOI
def extract_doi(work):
    work_dict=benedict.from_json(work)
    title=work_dict.get('work-summary[0].title.title.value')
    dois= [doi['external-id-value'] for doi in work_dict.get('work-summary[0].external-ids.external-id', []) if doi['external-id-type']=="doi"]
    # if there is a DOI assigned to the work, the list of dois is not empty and we can extract the first one
    doi=dois[0] if dois else None
    return doi, title


# ---- example execution
works=extract_works_section(orcid_record)
for work in works:
    doi,title = extract_doi(work)
    if doi:
        print(f"{doi}, {title}")

10.3389/frma.2021.694307, Enhancing Knowledge Graph Extraction and Validation From Scholarly Publications Using Bibliographic Metadata
10.3897/rio.7.e66264, OPTIMETA – Strengthening the Open Access publishing system through open citations and spatiotemporal metadata 
10.1080/19386389.2021.1999156, Roadmap to FAIR Research Information in Open Infrastructures
10.5281/zenodo.3900193, Research Profile Ownership through User Studies: A Case Study in the German National Research System
10.5281/zenodo.3896517, AEON - Towards an Academic Events Ontology
10.31263/voebm.v72i2.2808, Open Science und die Bibliothek – Aktionsfelder und Berufsbild
10.5281/zenodo.3242680, ROSI – Open Metrics for Open Repositories
10.5281/zenodo.3243485, Registry of [Open] Scientometric Data Sources – Technische Evaluierung von Offenen Datenquellen
10.1016/j.procs.2019.01.074, The Research Core Dataset (KDSF) in the Linked Data context
10.3897/rio.4.e31656, Reference implementation for open scientometric indicators (R